Здесь тот файл, что mdlj выдает будем приводить к моей csv-шке
---

In [80]:
import random
import os
import time

from numba import jit, njit, vectorize
import numpy as np
import scipy
from numpy.linalg import norm as norm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from tqdm import tqdm
import gc

from itertools import islice
from collections import defaultdict
import csv

In [81]:
# !rm *.csv *.xyz

In [82]:
!./md -N 50 -ns 5000 -outf 1 -thermof 1000 -dt 0.0005 -T0 1.0

# NVE MD Simulation of a Lennard - Jones fluid
# L = 7.368063 rho = 0.125 N = 50 r_cut = 1000000
# Steps number = 5000 seed = 1677869385 dt = 0.0005
# step time PE KE TE drift T
0 0 -17.311083 75 57.688917 0 1
1000 0.5 -38.278084 95.966762 57.688678 -4.1489319e-06 1.2795568
2000 1 -44.878223 102.56684 57.688621 -5.1355244e-06 1.3675579
3000 1.5 -41.13369 98.822419 57.688729 -3.2618371e-06 1.3176322
4000 2 -45.530925 103.21956 57.688636 -4.8681477e-06 1.3762608
5000 2.5 -47.744508 105.4331 57.688594 -5.595799e-06 1.4057747
7.368063


In [83]:
N = int(input())

In [84]:
# !./md -N 2 -ns 5000 -outf 1 -thermof 1000 -dt 0.0001 -T0 1.0

In [85]:
L = 2 * N ** (1 / 3)
L

7.368062997280773

In [86]:
read_path = f'N{N}.xyz'

In [87]:
# NUMPY VERSION:

def csv_from_xyz(read_path=read_path, velocity_regime=None, skip_part=0.5):
    '''

    '''
    # через решейп к (lines_read, 3) - удаляем с шагом строчки: x = np.delete(x, np.arange(0, x.size, use_particles))

    with open(read_path, 'r+') as read_f:
        all_vels = []
        all_coords = []
        lines_read = 0  # строчки с координатами прочитанные
        for line in (read_f):
            if line[0] != 'L' and len(line) > 5:    # ЗДЕСЬ 5 - ТИПО РАЗМЕР СТРОЧКИ ГДЕ КОЛ-ВО АТОМОВ ЗАПИСАНО - МОЖЕТ ОШИБКУ ВЫЗВАТЬ    
                # if lines_read in actual_steps:
                    # if lines_read % (N * step) == 0:    # делаем шаг

                    line = line[2:]  # ЭТО ТОЖЕ МОЖЕТ ВЫЗВАТЬ ОШИБКУ - ЕСЛИ НЕТ НАЧАЛЬНОГО "1" ИЛИ "C"
                    arr = list(map(lambda x: float(x.strip()), line.split(' ')))
                    arr_coords = (arr[:3])
                    arr_vels = (arr[3:])
                    all_vels.extend(arr_vels)
                    all_coords.extend(arr_coords)
                    lines_read += 1

    all_coords = np.reshape(all_coords, (lines_read // N, 3 * N))
    all_vels = np.reshape(all_vels, (lines_read // N, 3 * N))

    # CFG.N = N
    coords_path = 'coords' + str(N) + '.csv'
    velocities_path = 'velocities' + str(N) + '.csv'

    fieldnames_vels = []
    fieldnames_coords = []
    for i in range(N):
        fieldnames_coords.extend([str(i) + 'x', str(i) + 'y', str(i) + 'z'])

        fieldnames_vels.extend([str(i) + "v_x", str(i) + "v_y", str(i) + "v_z"])

    if not velocity_regime:
        df_coords = pd.DataFrame(all_coords,
                    index=np.arange(len(all_coords)),
                    columns=fieldnames_coords)[int(skip_part * len(all_coords)):]     # на старте там паника какая-то происходит - откидываем стартовые точки
        df_coords.index.name = 't'
        df_coords.to_csv(coords_path)

    df_vels = pd.DataFrame(all_vels,
                index=np.arange(len(all_vels)),
                columns=fieldnames_vels).iloc[int(skip_part * len(all_vels)):]
    df_vels.index.name = 't'
    df_vels.to_csv(velocities_path)

In [88]:
csv_from_xyz(skip_part=0.5)

In [89]:
@njit
def pot_energy(r1, r2):
    # Snaps particle back to the unfolded coordinate
    r1 = r1 % L
    r2 = r2 % L
    r = r1 - r2
    for i in range(3):
        if np.abs(r[i]) > L / 2:   # Если расстояние между двумя частицами больше L/2, то мы точно найдем в соседних клетках ближе
            r[i] = r[i] - L * np.sign(r[i])

    d = norm(r)
    return -4 * (pow(d, -6) - pow(d, -12))

def plot_energy(N=N, path_coords=f"coords{N}.csv", path_vels=f"velocities{N}.csv"):
    E = []
    try:
        coords = pd.read_csv(path_coords).drop(columns=["t"])
        vels = pd.read_csv(path_vels).drop(columns=["t"])
    except:
        coords = pd.read_csv(path_coords)
        vels = pd.read_csv(path_vels)
    
    for row_n in tqdm(range(len(vels))):  # TODO: ЗДЕСЬ СТОИТ vels ПОТОМУ ЧТО У МЕНЯ ПОКА БАГА И В КООРДС НА ОДНУ СТРОЧКУ БОЛЬШЕ ПОЧЕМУ-ТО...
        V = 0
        T = 0
        for i in range(N):
            r_i = np.array(coords.iloc[row_n, (3 * i):(3 * i) + 3])

            for j in range(i + 1, N):
                r_j = np.array(coords.iloc[row_n, (3 * j):(3 * j) + 3])
                # print(r_i, r_j)
                V += pot_energy(r_i, r_j)
        

        T = ((vels.iloc[row_n]) ** 2).sum() / 2
        E.append(T + V)

        # print(
        #     f'{row_n}: K = {T}, V = {V}, E = {T + V}'
        # )

    plt.figure(figsize=(10, 10))
    plt.plot(
        [i for i in range(len(E))], E
    )

    avg_E = np.array(E).mean()

    # print(E)

    plt.ylim([-0.1 + avg_E, 0.1 + avg_E])
    # plt.xlabel()

In [90]:
# создаем фолдед координаты:
df = pd.read_csv(f"coords{N}.csv") % L

df.to_csv(f"folded_coords{N}.csv", index=False)

In [91]:
plot_energy()

  1%|          | 18/2501 [00:02<05:28,  7.55it/s]


KeyboardInterrupt: 